# KNP( `-bnst` )ベースのチャンキング

文節内の連続する **名詞・接頭辞** に `B-NP` , `I-NP` をふる

- なんのための名詞句チャンキングか
  - キーワード抽出: 非構成的複合語の抽出
    - 教師データ必要
    - あるいは構成性を何らか定義する必要 (例. 構成性スコア `df(AB) / df(A)df(B)` )
  - 品詞区切りとしての名詞句（人間の字義通りの解釈を模倣）: 形式的定義でおおよそよい

- 考えられる形式的な名詞句の定義方法:
  - 0. 長い名詞句: 修飾要素も含めると保守的にとりすぎる
     - 「AのB」
     - 「連体修飾節A + B」
       - 「連体形複合辞（〜における、〜による）A + B」
  - 1. 文節内の連続する 名詞・接頭辞・<複合←> にふる（付属語を落とす）
     - 「AのB」「連体修飾節A + B」のような文節で区切られる名詞句が落ちる
     - 文節で区切られる「A B」名詞句のパターンを区別できる（ 業者|各数人 ）
     - 「バラク・オバマ」:（・: 付属語だが、<複合←> でとれる）

  - 2. 文節内の連続する <内容語><準内容語><複合←> にふる
     - 意味の弱い接頭辞・接尾辞が落ちる ( ご返事 -> 返事, 約半分 -> 半分, 加藤さん -> 加藤 )



cf. 文節の定義 := {begin: <文節始> の形態素, end: <文末>形態素 または 次の <文節始> が始まる前の形態素}
- 基本的に <接頭> （接頭辞，括弧始） が一番強い <文節始>
- 次いで <自立> が強い <文節始>
- 逆に <付属> は <文節始> になり得ない
- <複合←> も <文節始> になり得ない：<接頭>・<自立> が状況的に <文節始>になり得ないことを表す
  - Ｔ自立間連結 := 自立(→)，自立(←)にはさまれる 付属(←→)，接頭(←→)
  - (自立／Ｔ自立間連結(→)) (自立／Ｔ自立間連結(←)) の時，後者に<複合←>を与える

名詞

接頭辞

付属語
 - 助動詞
 - 接尾辞
 - 判定詞
 - 助詞
 - 特殊
   - (句点 読点 括弧終)
   - 各種記号
   - (＝ ・ ＆ ― ‐ … ‥ ―― …… ー 〜)
   - (↓ ↑ → ←)
   - (？ ！ ♪ ★ ☆)
   - (　)

In [1]:
# 'O'以外のラベルが含まれるようにBCCWJデータをフィルタ

filename = 'gsk-ene-1.1-bccwj-json-jumanpp-type/bccwj-ene-jumanpp-type.txt'
# def dataset_reader(filename):
with open(filename) as f:
    txt = f.read()
    sentences = []
    for sentence in txt.split('\n\n'):
        sentence_ = []
        __labels = []
        for line in sentence.split('\n'):
            if len(line.split(' ')) == 3:
                surface, pos, label = line.split(' ')
                sentence_.append({'surface': surface, 'pos': pos, 'label': label})
                __labels.append(label)
#             else:
#                 print(line)
        if sentence_ and len(set(__labels)) > 1:
            sentences.append(sentence_)
len(sentences)  # irex: 24750 <- 41057, type: 29497

29497

In [24]:
sentences[0]

[{'surface': '詰め', 'pos': '名詞', 'label': 'B-Product_Other'},
 {'surface': '将棋', 'pos': '名詞', 'label': 'I-Product_Other'},
 {'surface': 'の', 'pos': '助詞', 'label': 'O'},
 {'surface': '本', 'pos': '名詞', 'label': 'O'},
 {'surface': 'を', 'pos': '助詞', 'label': 'O'},
 {'surface': '買って', 'pos': '動詞', 'label': 'O'},
 {'surface': 'き', 'pos': '接尾辞', 'label': 'O'},
 {'surface': 'ました', 'pos': '接尾辞', 'label': 'O'},
 {'surface': '。', 'pos': '特殊', 'label': 'O'}]

In [51]:
from itertools import chain
# import pysnooper
from pyknp import KNP
from knp_base import KnpBase

knp = KNP(jumanpp=True, option='-bnst -tab')
knp = KnpBase(knp=KNP(jumanpp=True, option='-bnst -tab'))

def is_jiritsu(mrph):
#     print(mrph.fstring)
    return ('<内容語>' in mrph.fstring or '<準内容語>' in mrph.fstring or '<複合←>' in mrph.fstring) and mrph.hinsi in {'名詞', '接尾辞', '接頭辞', '特殊'}

def extract_jiritsu(bnst):
    return [is_jiritsu(m) for m in bnst.mrph_list()]


def annotate_np(blist):
    flags = chain.from_iterable([extract_jiritsu(b) for b in blist])
    tags = []
    current_tag = 'O'
    for f in flags:
        if not f:
            tags.append('O')
            current_tag = 'O'
        elif current_tag == 'O':
            tags.append('B-NP')
            current_tag = 'B-NP'
        else:
            tags.append('I-NP')
            
    return tags


def annotate_np_from_tokens(tokens):
    text = ''.join(tokens)
    np_tags_sentence = []
    for sentence in text.split('。'):
        blist =knp.parse(sentence)
        np_tags = annotate_np(blist)

        np_tags_sentence.append(' '.join(np_tags))
    np_tags = ' O '.join(np_tags_sentence)
    if np_tags:
        np_tags = (np_tags + ' O' if text.endswith('。') else np_tags).split(' ')
        if len(tokens) != len(np_tags):
            print(len(tokens), len(np_tags))
            print( '\n'.join(f'{token} {tag}' for token, tag in zip(tokens, np_tags)) )
            print(text)
    return np_tags

annotate_np_from_tokens(['「', 'マイケル', '・', 'フェイ', '君', '」', 'の', '仕事'])

['O', 'B-NP', 'I-NP', 'I-NP', 'O', 'O', 'O', 'B-NP']

In [ ]:
for sentence in sentences:
    tokens = [d['surface'] for d in sentence]
    np_tags = annotate_np_from_tokens(tokens)
    for np_tag, d in zip(np_tags, sentence):
        d['chunk'] = np_tag
        
# sentences

In [40]:
s = '（金八先生に、ジャニーズ事務所の有望タレントが出演している事実は、ご存知ですよね）。今日の苗字ランキングの番組で、実際あった苗字ってこの中のどれだったんですか？七月一日　八月一日　九月一日　十月一日裏番組に熱中して見逃しちゃいました。'
knp.wakati(s)

'（ 金 八 先生 に 、 ジャニーズ 事務 所 の 有望 タレント が 出演 して いる 事実 は 、 ご存知 ですよね ） 。 今日 の 苗字 ランキング の 番組 で 、 実際 あった 苗字 って この 中 の どれ だった んです か ？ 七 月 一 日 \u3000 八 月 一 日 \u3000 九 月 一 日 \u3000 十 月 一 日 裏 番組 に 熱中 して 見逃しちゃ い ました 。'